# GAT — Graph Construction
Transforms IMPECT event data into per-possession `torch_geometric.data.Data` objects for a Graph Attention Network.

**Graph design:**
- **Node** = one action (row) in a possession sequence
- **Real edge** i → i+1: the sequential action transition
- **Synthetic edge** i → i+2 and i → i+3: long-range synergy links

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import LabelEncoder
from kloppy import impect
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
# Load the full event stream so we can detect possession changes correctly
MATCH_ID       = 122838
COMPETITION_ID = 743

dataset = impect.load_open_data(match_id=MATCH_ID, competition_id=COMPETITION_ID)

events_df = (
    dataset
    .transform(to_coordinate_system="secondspectrum")
    .to_df(engine="pandas")
)

# Use the full event stream — NO_VIDEO events are handled at graph-build time
df = events_df.copy().reset_index(drop=True)

print(f"Total events: {len(df)}")
print(f"Event types:  {sorted(df['event_type'].unique().tolist())}")
print(f"Columns:      {df.columns.tolist()}")

In [ ]:
def parse_duration_to_seconds(ts) -> float:
    """
    Convert a Kloppy duration value to float seconds.
    Handles pd.Timedelta objects and string formats such as:
      '0\u00b5s', '4s 192999\u00b5s', '50m 29s 618999\u00b5s', '24s 350ms', '50m 39s 229ms'
    """
    if ts is None or (isinstance(ts, float) and np.isnan(ts)):
        return 0.0
    if isinstance(ts, pd.Timedelta):
        return ts.total_seconds()
    total = 0.0
    for part in str(ts).split():
        if   part.endswith("\u00b5s"):  total += float(part[:-2]) / 1_000_000
        elif part.endswith("ms"):        total += float(part[:-2]) / 1_000
        elif part.endswith("m"):         total += float(part[:-1]) * 60
        elif part.endswith("s"):         total += float(part[:-1])
    return round(total, 6)


# Sanity tests
assert abs(parse_duration_to_seconds("0\u00b5s")                - 0.0)        < 1e-9
assert abs(parse_duration_to_seconds("4s 192999\u00b5s")        - 4.192999)   < 1e-6
assert abs(parse_duration_to_seconds("50m 29s 618999\u00b5s")   - 3029.618999) < 1e-4
assert abs(parse_duration_to_seconds("24s 350ms")                - 24.350)     < 1e-6
assert abs(parse_duration_to_seconds("50m 39s 229ms")            - 3039.229)   < 1e-4
print("Timestamp parser OK")

In [ ]:
# --- NaN fills ---
# team_id: forward-fill so events like NO_VIDEO inherit surrounding team context
df["team_id"]           = df["team_id"].ffill()
# player_id: some events (e.g. NO_VIDEO) have no player
df["player_id"]         = df["player_id"].fillna("UNKNOWN")
df["pass_type"]         = df["pass_type"].fillna("NONE")
df["is_under_pressure"] = df["is_under_pressure"].fillna(False).astype(bool)
df["result"]            = df["result"].fillna("NONE")
df["success"]           = df["success"].fillna(False).astype(bool)
# Fill start coords before using them to fill end coords
df["coordinates_x"]     = df["coordinates_x"].fillna(0.0)
df["coordinates_y"]     = df["coordinates_y"].fillna(0.0)
df["end_coordinates_x"] = df["end_coordinates_x"].fillna(df["coordinates_x"])
df["end_coordinates_y"] = df["end_coordinates_y"].fillna(df["coordinates_y"])

# --- Parse timestamps and sort chronologically ---
df["timestamp_sec"] = df["timestamp"].apply(parse_duration_to_seconds)
df = df.sort_values(["period_id", "timestamp_sec"]).reset_index(drop=True)

# --- Fit encoders on the full match (globally consistent IDs across all graphs) ---
player_enc     = LabelEncoder().fit(df["player_id"].astype(str))
event_type_enc = LabelEncoder().fit(df["event_type"].astype(str))
pass_type_enc  = LabelEncoder().fit(df["pass_type"].astype(str))
result_enc     = LabelEncoder().fit(df["result"].astype(str))

df["player_id_enc"]   = player_enc.transform(df["player_id"].astype(str))
df["event_type_enc"]  = event_type_enc.transform(df["event_type"].astype(str))
df["pass_type_enc"]   = pass_type_enc.transform(df["pass_type"].astype(str))
df["result_enc"]      = result_enc.transform(df["result"].astype(str))

print(f"Players:     {len(player_enc.classes_)} unique")
print(f"Event types: {list(event_type_enc.classes_)}")
print(f"Pass types:  {list(pass_type_enc.classes_)}")
print(f"Results:     {list(result_enc.classes_)}")

In [ ]:
def segment_possessions(df: pd.DataFrame) -> list[pd.DataFrame]:
    """
    Split a match DataFrame (all events, sorted chronologically) into
    possession sequences.

    A new possession begins when:
      - team_id changes from the previous event, OR
      - the previous event was a SHOT

    Returns a list of DataFrames each with a reset 0-based index.
    """
    possessions   = []
    current_start = 0

    for i in range(1, len(df)):
        prev = df.iloc[i - 1]
        curr = df.iloc[i]

        if prev["team_id"] != curr["team_id"] or prev["event_type"] == "SHOT":
            possessions.append(df.iloc[current_start:i].reset_index(drop=True))
            current_start = i

    possessions.append(df.iloc[current_start:].reset_index(drop=True))
    return possessions


possessions = segment_possessions(df)
lengths     = [len(p) for p in possessions]

print(f"Possessions: {len(possessions)}")
print(f"Size — min: {min(lengths)}, max: {max(lengths)}, mean: {np.mean(lengths):.1f}")

In [ ]:
import math

# --- Score constants ---
FIELD_HALF        = 52.5          # x-axis half-length (m)
FIELD_LENGTH      = 105.0         # total pitch length (m)
FIELD_WIDTH       = 68.0          # total pitch width (m)
# Max distance any point on the field can be from a goal (corner-to-goal diagonal)
MAX_FIELD_DIST    = (FIELD_LENGTH ** 2 + (FIELD_WIDTH / 2) ** 2) ** 0.5  # ≈ 110.4 m
SIGMOID_SHIFT     = 7.5           # sigmoid midpoint — gives ≈ 0 at t=0, ≈ 1 at t=15 s
ADVANCEMENT_WEIGHT = 1.0          # scales the linear position-from-center-field term

# End-of-play event bonuses
GOAL_BONUS        =  1.0
SHOT_BONUS        =  0.3
OUT_OF_BOUNDS_PEN = -0.1
INTERCEPT_PEN     = -0.2


def get_attacking_goal_x(team_id: str, period_id: int, home_team_id: str) -> float:
    """
    Return the x-coordinate of the opponent's goal for the possessing team.

    Convention assumed for Kloppy secondspectrum + IMPECT Bundesliga data:
      Period 1 — home team attacks toward +x  (opponent's goal at x = +52.5)
      Period 2 — teams switch; home attacks toward -x (opponent's goal at x = -52.5)
    """
    is_home = str(team_id) == str(home_team_id)
    attacks_positive = (is_home and period_id == 1) or (not is_home and period_id == 2)
    return FIELD_HALF if attacks_positive else -FIELD_HALF


def score_play(play_clean: pd.DataFrame, attacking_goal_x: float) -> float:
    """
    Compute scalar play score P for one possession sequence (NO_VIDEO removed).

        P = (displacement_scaled * advancement_factor * sigmoid_t) + end_bonus

    Components
    ----------
    displacement_scaled : float [-1, 1]
        How much the ball moved toward the opponent's goal during the play,
        normalised by MAX_FIELD_DIST (~110.4 m). Negative when the ball moved
        away from goal, positive when it moved toward goal.

    advancement_factor : float [0, 1]
        ADVANCEMENT_WEIGHT × (x_end projected onto the attack axis, normalised
        by FIELD_HALF). 0 = ball at or behind the halfway line, 1 = ball at
        the opponent's goal line. Linear reward for finishing deep in the
        attacking half.

    sigmoid_t : float [0, 1]
        Logistic sigmoid of (t - 7.5), where t is the play duration in seconds.
        Starts near 0 at t = 0 and approaches 1 at t = 15 s, rewarding
        sustained possession.

    end_bonus : float
        SHOT resulting in GOAL  : +1.0
        SHOT (no goal)          : +0.3
        Ball out of bounds      : -0.1   (last result contains 'OUT')
        Interception            : -0.2   (last event_type == 'INTERCEPTION')
        Otherwise               :  0.0

    Parameters
    ----------
    play_clean : pd.DataFrame
        Possession rows with NO_VIDEO removed, sorted chronologically.
    attacking_goal_x : float
        x-coordinate of the opponent's goal (+52.5 or -52.5).

    Returns
    -------
    float : P
    """
    if len(play_clean) == 0:
        return 0.0

    first = play_clean.iloc[0]
    last  = play_clean.iloc[-1]

    x_start = float(first["coordinates_x"])
    y_start = float(first["coordinates_y"])
    x_end   = float(last["end_coordinates_x"])
    y_end   = float(last["end_coordinates_y"])
    goal_y  = 0.0

    # --- 1. Ball displacement toward goal [-1, 1] ---
    d_start = ((x_start - attacking_goal_x) ** 2 + (y_start - goal_y) ** 2) ** 0.5
    d_end   = ((x_end   - attacking_goal_x) ** 2 + (y_end   - goal_y) ** 2) ** 0.5
    displacement_scaled = min(1.0, (d_start - d_end) / MAX_FIELD_DIST)

    # --- 2. Linear distance from centre-field in the attacking direction [0, 1] ---
    # 0 = ball at or behind the halfway line, 1 = ball at the opponent's goal line
    attack_sign          = 1.0 if attacking_goal_x > 0 else -1.0
    position_from_center = max(0.0, min(1.0, x_end * attack_sign / FIELD_HALF))
    advancement_factor   = ADVANCEMENT_WEIGHT * position_from_center

    # --- 3. Sigmoid of play duration: ≈ 0 at t=0, ≈ 1 at t=15 s ---
    t     = float(last["timestamp_sec"] - first["timestamp_sec"])
    sig_t = 1.0 / (1.0 + math.exp(-(t - SIGMOID_SHIFT)))

    mult = displacement_scaled * advancement_factor * sig_t

    # --- 4. End-of-play event bonus ---
    last_type   = str(last["event_type"])
    last_result = str(last["result"]).upper()

    if last_type == "SHOT":
        end_bonus = GOAL_BONUS if "GOAL" in last_result else SHOT_BONUS
    elif "OUT" in last_result:
        end_bonus = OUT_OF_BOUNDS_PEN
    elif last_type == "INTERCEPTION":
        end_bonus = INTERCEPT_PEN
    else:
        end_bonus = 0.0

    return mult + end_bonus

In [ ]:
NO_VIDEO_THRESHOLD = 3.0  # seconds — discard a play if NO_VIDEO exceeds this
N_SYNTHETIC_EDGES  = 2    # how many synthetic long-range hops to add per node
                          # k=2 → i→i+2, k=3 → i→i+3, etc.


def build_graph(play: pd.DataFrame, home_team_id: str) -> Data | None:
    """
    Build a PyTorch Geometric Data object from one possession sequence.

    NO_VIDEO handling:
      - Duration of each event = gap to the next event's timestamp.
      - If the total NO_VIDEO duration in the play exceeds NO_VIDEO_THRESHOLD,
        the entire play is discarded (return None).
      - Otherwise, NO_VIDEO rows are removed before building the graph.

    Node features (6):
      [player_id_enc, coord_x, coord_y, event_type_enc, timestamp_rel, is_under_pressure]

    Edge attributes (7):
      [edge_type, pass_type_enc, end_x, end_y, action_distance, result_enc, success]
      edge_type       : 1.0 = real sequential edge, 2.0 = synthetic long-range edge
      action_distance : for real edges — Euclidean distance (metres);
                        for synthetic edges — hop proximity weight = 1/k
                        (skip-1 k=2 → 0.5, skip-2 k=3 → 0.33, …)

    g.y          : tensor([P]) — play score from score_play()
    g.player_ids : list[str]   — player_id for each node (parallel to node rows)
    g.team_id    : str         — possession team
    g.period_id  : int         — period the play occurs in
    g.t_start    : float       — timestamp_sec of the first event in the play
    """
    # --- NO_VIDEO check ---
    timestamps = play["timestamp_sec"].to_numpy()
    durations  = np.diff(timestamps)
    durations  = np.append(durations, 0.0)

    no_video_mask = (play["event_type"] == "NO_VIDEO").to_numpy()
    if no_video_mask.any():
        if durations[no_video_mask].sum() > NO_VIDEO_THRESHOLD:
            return None  # too much missing video — discard play

    # Remove NO_VIDEO rows; the remaining events form the graph
    play_clean = play[~no_video_mask].reset_index(drop=True)

    n = len(play_clean)
    if n < 2:
        return None

    play_start_sec = play_clean["timestamp_sec"].iloc[0]

    # --- Node features (N, 6) ---
    node_features = [
        [
            float(row["player_id_enc"]),
            float(row["coordinates_x"]),
            float(row["coordinates_y"]),
            float(row["event_type_enc"]),
            float(row["timestamp_sec"] - play_start_sec),
            float(row["is_under_pressure"]),
        ]
        for _, row in play_clean.iterrows()
    ]
    x = torch.tensor(node_features, dtype=torch.float)

    # --- Edges ---
    src, dst, attrs = [], [], []
    REAL_TYPE  = 1.0
    SYNTH_TYPE = 2.0

    for i in range(n):
        row = play_clean.iloc[i]
        x1, y1 = row["coordinates_x"],    row["coordinates_y"]
        x2, y2 = row["end_coordinates_x"], row["end_coordinates_y"]
        dist    = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

        real_attr = [
            REAL_TYPE,
            float(row["pass_type_enc"]),
            float(x2), float(y2),
            float(dist),
            float(row["result_enc"]),
            float(row["success"]),
        ]

        if i + 1 < n:  # real sequential edge
            src.append(i); dst.append(i + 1); attrs.append(real_attr)
        for k in range(2, 2 + N_SYNTHETIC_EDGES):  # synthetic long-range hops
            if i + k < n:
                hop_weight = 1.0 / k  # 0.5 for skip-1, 0.33 for skip-2, …
                synth_attr = [SYNTH_TYPE, 0.0, 0.0, 0.0, hop_weight, 0.0, 0.0]
                src.append(i); dst.append(i + k); attrs.append(synth_attr)

    edge_index = torch.tensor([src, dst], dtype=torch.long)  # (2, E)
    edge_attr  = torch.tensor(attrs, dtype=torch.float)       # (E, 7)

    # --- Play score (training label) ---
    goal_x = get_attacking_goal_x(
        play_clean.iloc[0]["team_id"],
        play_clean.iloc[0]["period_id"],
        home_team_id,
    )
    y = torch.tensor([score_play(play_clean, goal_x)], dtype=torch.float)

    # --- Assemble Data object with metadata for lineup / interaction matrix ---
    g = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    g.player_ids = play_clean["player_id"].tolist()          # list[str], len == n nodes
    g.team_id    = str(play_clean.iloc[0]["team_id"])
    g.period_id  = int(play_clean.iloc[0]["period_id"])
    g.t_start    = float(play_clean.iloc[0]["timestamp_sec"])
    return g

In [ ]:
home_team_id = str(dataset.metadata.teams[0].team_id)

graphs  = []
skipped = 0

for play in possessions:
    g = build_graph(play, home_team_id)
    if g is None:
        skipped += 1
    else:
        graphs.append(g)

scores = [g.y.item() for g in graphs]
print(f"Graphs built:  {len(graphs)}")
print(f"Skipped:       {skipped}  (single-event plays or NO_VIDEO > {NO_VIDEO_THRESHOLD}s)")
print(f"Score — min: {min(scores):.3f}, max: {max(scores):.3f}, mean: {np.mean(scores):.3f}")

In [ ]:
def validate_graphs(graphs: list[Data]) -> None:
    sizes = [g.num_nodes for g in graphs]
    edges = [g.num_edges for g in graphs]

    print(f"Total graphs: {len(graphs)}")
    print(f"Nodes — min: {min(sizes)}, max: {max(sizes)}, mean: {np.mean(sizes):.1f}")
    print(f"Edges — min: {min(edges)}, max: {max(edges)}, mean: {np.mean(edges):.1f}")

    for i, g in enumerate(graphs):
        assert g.x.shape[1] == 6,          f"Graph {i}: node feature width {g.x.shape[1]} != 6"
        assert g.edge_attr.shape[1] == 7,  f"Graph {i}: edge attr width {g.edge_attr.shape[1]} != 7"
        assert g.edge_index.shape[0] == 2, f"Graph {i}: edge_index shape {g.edge_index.shape}"
        assert g.edge_index.max().item() < g.num_nodes, f"Graph {i}: out-of-range node index"
        assert g.y.shape == (1,),          f"Graph {i}: y shape {g.y.shape} != (1,)"

        n = g.num_nodes
        expected = max(0, n - 1) + sum(max(0, n - k) for k in range(2, 2 + N_SYNTHETIC_EDGES))
        assert g.num_edges == expected, (
            f"Graph {i}: expected {expected} edges for n={n}, N_SYNTHETIC_EDGES={N_SYNTHETIC_EDGES}, got {g.num_edges}"
        )

    print("All assertions passed.")

    print("\nSample graphs:")
    for g in graphs[:5]:
        real  = (g.edge_attr[:, 0] == 1.0).sum().item()
        synth = (g.edge_attr[:, 0] == 2.0).sum().item()
        print(f"  nodes={g.num_nodes:>3}, edges={g.num_edges:>3} "
              f"(real={real:>2}, synthetic={synth:>2})  score={g.y.item():+.4f}")


validate_graphs(graphs)

In [ ]:
class PlayGAT(nn.Module):
    """
    Two-layer Graph Attention Network that predicts a scalar play score.
    Uses edge features (edge_dim=7) in both attention layers.

    Architecture
    ------------
    GATConv(6 → 32, heads=4, concat=True,  edge_dim=7)  →  128-dim per node
    BatchNorm1d(128) + ELU + Dropout
    GATConv(128 → 32, heads=1, concat=False, edge_dim=7) →  32-dim per node
    ELU
    global_mean_pool                                      →  32-dim per graph
    Linear(32, 1)                                         →  scalar play score

    Notes
    -----
    add_self_loops=False is required because the graphs were built without
    self-loops; PyG's default (True) would try to merge edge_attr with
    auto-added self-loop rows, causing a shape mismatch.
    """

    NODE_DIM = 6   # must match g.x.shape[1]
    EDGE_DIM = 7   # must match g.edge_attr.shape[1]

    def __init__(self, hidden: int = 32, heads: int = 4, dropout: float = 0.3):
        super().__init__()
        self.dropout = dropout

        self.conv1 = GATConv(
            self.NODE_DIM, hidden, heads=heads, concat=True,
            edge_dim=self.EDGE_DIM, dropout=dropout, add_self_loops=False,
        )
        self.bn1 = nn.BatchNorm1d(hidden * heads)  # 128

        self.conv2 = GATConv(
            hidden * heads, hidden, heads=1, concat=False,
            edge_dim=self.EDGE_DIM, dropout=dropout, add_self_loops=False,
        )

        self.head = nn.Linear(hidden, 1)

    def forward(self, x, edge_index, edge_attr, batch):
        x = self.conv1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(self.bn1(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.elu(self.conv2(x, edge_index, edge_attr=edge_attr))
        x = global_mean_pool(x, batch)          # (batch_size, 32)
        return self.head(x).squeeze(-1)          # (batch_size,)

    def forward_with_attention(self, data):
        """
        Single-graph forward pass returning per-layer attention weights.
        Useful for post-hoc edge attribution.

        Returns
        -------
        pred  : Tensor (1,)
        attn1 : Tensor (E, heads)  — Layer 1 per-head attention weights
        attn2 : Tensor (E, 1)       — Layer 2 attention weights
        """
        x, ei, ea = data.x, data.edge_index, data.edge_attr
        batch = torch.zeros(x.size(0), dtype=torch.long)

        x, (_, attn1) = self.conv1(x, ei, edge_attr=ea, return_attention_weights=True)
        x = F.elu(self.bn1(x))
        x, (_, attn2) = self.conv2(x, ei, edge_attr=ea, return_attention_weights=True)
        x = F.elu(x)
        x = global_mean_pool(x, batch)
        return self.head(x).squeeze(-1), attn1, attn2


# --- Instantiate and sanity-check ---
model = PlayGAT(hidden=32, heads=4, dropout=0.3)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(model)
print(f"\nTrainable parameters: {total_params:,}")

with torch.no_grad():
    g0 = graphs[0]
    dummy_batch = torch.zeros(g0.num_nodes, dtype=torch.long)
    out = model(g0.x, g0.edge_index, g0.edge_attr, dummy_batch)
    print(f"\nForward check  — nodes: {g0.num_nodes}, output: {out.shape}, value: {out.item():.4f}")
    pred, a1, a2 = model.forward_with_attention(g0)
    print(f"Attention check — attn1: {a1.shape}, attn2: {a2.shape}")

In [ ]:
# ── Training configuration ───────────────────────────────────────────────────
SEED         = 42
BATCH_SIZE   = 32
LR           = 1e-3
WEIGHT_DECAY = 1e-4
N_EPOCHS     = 50
LOG_EVERY    = 10

torch.manual_seed(SEED)

# ── 80 / 20 reproducible split ────────────────────────────────────────────────
perm    = torch.randperm(len(graphs), generator=torch.Generator().manual_seed(SEED)).tolist()
n_train = int(0.8 * len(graphs))

train_graphs = [graphs[i] for i in perm[:n_train]]
val_graphs   = [graphs[i] for i in perm[n_train:]]

train_loader = DataLoader(train_graphs, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_graphs,   batch_size=BATCH_SIZE, shuffle=False)

print(f"Train: {len(train_graphs)} graphs  |  Val: {len(val_graphs)} graphs")

# ── Model, optimiser, loss ────────────────────────────────────────────────────
model     = PlayGAT(hidden=32, heads=4, dropout=0.3)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

train_losses: list[float] = []
val_losses:   list[float] = []

# ── Training loop ─────────────────────────────────────────────────────────────
for epoch in range(1, N_EPOCHS + 1):
    model.train()
    epoch_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        pred = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        loss = criterion(pred, batch.y.squeeze(-1))   # batch.y is (B,1); pred is (B,)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * batch.num_graphs
    avg_train = epoch_loss / len(train_graphs)
    train_losses.append(avg_train)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            pred     = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            val_loss += criterion(pred, batch.y.squeeze(-1)).item() * batch.num_graphs
    avg_val = val_loss / len(val_graphs)
    val_losses.append(avg_val)

    if epoch % LOG_EVERY == 0 or epoch == 1:
        print(f"Epoch {epoch:>3}/{N_EPOCHS}  |  train MSE: {avg_train:.4f}  |  val MSE: {avg_val:.4f}")

print("\nTraining complete.")

# ── Loss curve ────────────────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(range(1, N_EPOCHS + 1), train_losses, label="Train MSE")
ax.plot(range(1, N_EPOCHS + 1), val_losses,   label="Val MSE",   linestyle="--")
ax.set_xlabel("Epoch")
ax.set_ylabel("MSE Loss")
ax.set_title("PlayGAT Training Curve")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
from collections import defaultdict

# ── Player name lookup ────────────────────────────────────────────────────────
player_id_to_name: dict[str, str] = {}
for _team in dataset.metadata.teams:
    for _p in _team.players:
        player_id_to_name[str(_p.player_id)] = _p.name

# ── Starting lineups from Kloppy metadata ────────────────────────────────────
# player.starting is confirmed True/False in IMPECT open data
_team_starters: dict[str, frozenset[str]] = {}
for _team in dataset.metadata.teams:
    tid      = str(_team.team_id)
    starters = frozenset(str(_p.player_id) for _p in _team.players if _p.starting)
    _team_starters[tid] = starters
    names = [player_id_to_name.get(pid, pid) for pid in sorted(starters)]
    print(f"Team {tid} starters ({len(starters)}): {names}")

# ── Parse SUBSTITUTION events ─────────────────────────────────────────────────
_sub_events = df[df["event_type"] == "SUBSTITUTION"].copy()
print(f"\nSubstitution events: {len(_sub_events)}")

_has_receiver = (
    "receiver_player_id" in df.columns
    and _sub_events["receiver_player_id"].notna().any()
)
print(f"receiver_player_id populated for subs: {_has_receiver}")

# sub_log[team_id] = sorted list of (period_id, timestamp_sec, off_id, on_id)
_sub_log: dict[str, list[tuple[int, float, str, str]]] = defaultdict(list)

if _has_receiver and len(_sub_events) > 0:
    for _, row in _sub_events.iterrows():
        tid    = str(row["team_id"])
        period = int(row["period_id"])
        t_sec  = float(row["timestamp_sec"])
        off_id = str(row["player_id"])
        on_id  = str(row.get("receiver_player_id", float("nan")))
        if on_id not in ("nan", "UNKNOWN", "None"):
            _sub_log[tid].append((period, t_sec, off_id, on_id))
    for tid, subs in _sub_log.items():
        subs.sort(key=lambda x: (x[0], x[1]))
        print(f"  Team {tid}: {len(subs)} substitution(s) tracked")
else:
    print("  Falling back to period-appearance heuristic.")

_sub_log_frozen       = dict(_sub_log)
_team_starters_frozen = dict(_team_starters)


def get_lineup_at_time(team_id: str, period_id: int, timestamp_sec: float) -> frozenset[str]:
    """
    Return the set of player IDs on the field for `team_id` at the given time.

    Primary path (when substitution data is available):
      Starts from the team's starting XI, applies any subs that occurred at
      or before (period_id, timestamp_sec) in chronological order.

    Fallback path (no sub data):
      Returns all players who appeared in any event for this team in this
      period up to this timestamp (safe over-approximation).

    Always excludes "UNKNOWN".
    """
    tid = str(team_id)

    if _sub_log_frozen:
        lineup = set(_team_starters_frozen.get(tid, frozenset()))
        for s_period, s_t, off_id, on_id in _sub_log_frozen.get(tid, []):
            if (s_period, s_t) <= (period_id, timestamp_sec):
                lineup.discard(off_id)
                lineup.add(on_id)
        lineup.discard("UNKNOWN")
        return frozenset(lineup)
    else:
        mask = (
            (df["team_id"].astype(str) == tid) &
            (
                (df["period_id"] < period_id) |
                ((df["period_id"] == period_id) & (df["timestamp_sec"] <= timestamp_sec))
            ) &
            (df["player_id"] != "UNKNOWN")
        )
        players = set(df[mask]["player_id"].unique())
        players.discard("UNKNOWN")
        return frozenset(players)


# ── Sanity check ──────────────────────────────────────────────────────────────
for _team in dataset.metadata.teams:
    tid = str(_team.team_id)
    lu  = get_lineup_at_time(tid, 1, 0.0)
    print(f"\nTeam {tid} at kick-off ({len(lu)} players):")
    for pid in sorted(lu):
        print(f"  {pid}: {player_id_to_name.get(pid, '???')}")

In [ ]:
# ── Step 1: Model inference on every graph ───────────────────────────────────
# Iterate one graph at a time (DataLoader not used here because g.player_ids /
# g.team_id / g.period_id / g.t_start are non-tensor attributes that are not
# collated by PyG's Batch).
model.eval()
predicted_scores: list[float] = []
with torch.no_grad():
    for g in graphs:
        dummy_batch = torch.zeros(g.num_nodes, dtype=torch.long)
        pred = model(g.x, g.edge_index, g.edge_attr, dummy_batch)
        predicted_scores.append(pred.item())

print(f"Inference complete: {len(predicted_scores)} graphs")
print(f"Predicted score range: [{min(predicted_scores):.3f}, {max(predicted_scores):.3f}]")

# ── Step 2: Build player universe (both teams, excluding UNKNOWN) ─────────────
all_pids: list[str] = sorted(
    {pid for g in graphs for pid in g.player_ids if pid != "UNKNOWN"}
)
n_players  = len(all_pids)
pid_to_idx = {pid: i for i, pid in enumerate(all_pids)}
away_team_id = str(dataset.metadata.teams[1].team_id)
print(f"Unique players in graphs: {n_players}")

# ── Step 3: Accumulate statistics ─────────────────────────────────────────────
# score_sum[i, j]          — weighted sum of predicted scores over plays where i & j interact
# participation_count[i]   — # plays player i appeared in as a node
# field_plays[i]           — # plays during which player i was on the field
# field_pairs[i, j]        — # plays during which both i and j were on field

score_sum           = np.zeros((n_players, n_players), dtype=np.float64)
participation_count = np.zeros(n_players,               dtype=np.float64)
field_plays         = np.zeros(n_players,               dtype=np.float64)
field_pairs         = np.zeros((n_players, n_players),  dtype=np.float64)

for g, pred_score in zip(graphs, predicted_scores):
    # ── Full on-field lineup for this play (both teams) ───────────────────
    opp_tid     = away_team_id if g.team_id == home_team_id else home_team_id
    lineup_poss = get_lineup_at_time(g.team_id, g.period_id, g.t_start)
    lineup_opp  = get_lineup_at_time(opp_tid,   g.period_id, g.t_start)
    full_lineup = lineup_poss | lineup_opp
    lu_indices  = [pid_to_idx[pid] for pid in full_lineup if pid in pid_to_idx]

    # field_plays denominator
    for li in lu_indices:
        field_plays[li] += 1.0

    # field_pairs denominator (symmetric)
    for ii, li in enumerate(lu_indices):
        for lj in lu_indices[ii:]:
            field_pairs[li, lj] += 1.0
            if li != lj:
                field_pairs[lj, li] += 1.0

    # ── Diagonal: each unique participating player ─────────────────────────
    participating = {pid_to_idx[pid] for pid in g.player_ids if pid in pid_to_idx}
    for pi in participating:
        score_sum[pi, pi]       += pred_score
        participation_count[pi] += 1.0

    # ── Off-diagonal: edge-linked player PAIRS (hop-proximity weighted) ────
    # Real edges (edge_type=1.0) get weight 1.0.
    # Synthetic skip-k edges store hop proximity weight 1/k in action_distance (index 4).
    # If a pair is connected by multiple edges, keep the highest weight (max → real wins).
    pair_weights: dict[tuple[int, int], float] = {}
    for e in range(g.edge_index.shape[1]):
        src_pid = g.player_ids[g.edge_index[0, e].item()]
        dst_pid = g.player_ids[g.edge_index[1, e].item()]
        if src_pid == "UNKNOWN" or dst_pid == "UNKNOWN":
            continue
        if src_pid not in pid_to_idx or dst_pid not in pid_to_idx:
            continue
        pi, pj = pid_to_idx[src_pid], pid_to_idx[dst_pid]
        if pi == pj:
            continue
        pair   = (min(pi, pj), max(pi, pj))
        weight = 1.0 if g.edge_attr[e, 0].item() == 1.0 else g.edge_attr[e, 4].item()
        pair_weights[pair] = max(pair_weights.get(pair, 0.0), weight)

    for (pi, pj), w in pair_weights.items():
        score_sum[pi, pj] += pred_score * w
        score_sum[pj, pi] += pred_score * w

# ── Step 4: Build the interaction matrix ──────────────────────────────────────
#
# Off-diagonal [i,j]:
#   = weighted_score_sum[i,j] / field_pairs[i,j]
#   Weighting by hop proximity dampens inflation from distant synthetic connections.
#
# Diagonal [i,i]:
#   = avg_predicted_score_when_player_participates  ×  P(participates | on field)
#   = score_sum[i,i] / field_plays[i]

interaction_matrix = np.zeros((n_players, n_players), dtype=np.float64)

with np.errstate(invalid="ignore", divide="ignore"):
    off = np.where(field_pairs > 0, score_sum / field_pairs, 0.0)
    np.fill_diagonal(off, 0.0)
    interaction_matrix += off

    diag_vals = np.where(field_plays > 0, np.diag(score_sum) / field_plays, 0.0)
    np.fill_diagonal(interaction_matrix, diag_vals)

# ── Step 5a: DataFrame display ────────────────────────────────────────────────
player_names   = [player_id_to_name.get(pid, pid) for pid in all_pids]
interaction_df = pd.DataFrame(interaction_matrix, index=player_names, columns=player_names)

print("\nPlayer Interaction Matrix (top-left 8 × 8):")
with pd.option_context("display.float_format", "{:.3f}".format, "display.max_columns", 8):
    print(interaction_df.iloc[:8, :8])

# ── Step 5b: Heatmap ──────────────────────────────────────────────────────────
fig_size = max(10, n_players * 0.55)
fig, ax  = plt.subplots(figsize=(fig_size, fig_size * 0.85))

vmax = np.abs(interaction_matrix).max() or 1.0
norm = mcolors.TwoSlopeNorm(vmin=-vmax, vcenter=0.0, vmax=vmax)
im   = ax.imshow(interaction_matrix, cmap="RdYlGn", norm=norm, aspect="auto")

ax.set_xticks(range(n_players))
ax.set_yticks(range(n_players))
ax.set_xticklabels(player_names, rotation=90, fontsize=7)
ax.set_yticklabels(player_names, fontsize=7)

plt.colorbar(im, ax=ax, label="hop-weighted avg predicted score / field-time probability")
ax.set_title(
    f"Player Interaction Matrix — Match {MATCH_ID}\n"
    "Diagonal: avg score × P(participate | on field)   "
    "Off-diagonal: hop-weighted avg score × P(linked by edge | both on field)",
    fontsize=9,
)
plt.tight_layout()
plt.show()

print(f"\nMatrix shape: {interaction_matrix.shape}")
print(f"Value range:  [{interaction_matrix.min():.4f}, {interaction_matrix.max():.4f}]")